<a href="https://colab.research.google.com/github/0Kyumy0/Machine_Learning_CodingDojo/blob/Other_Topics/featureEngineeringExercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [123]:
import pandas as pd
import numpy as np

from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn import set_config
set_config(display='diagram')

In [ ]:
filename = '/content/drive/MyDrive/DataScience/BasesDeDatos/bikeshare_train - bikeshare_train.csv'
df = pd.read_csv(filename)
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 0:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 1:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 2:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 3:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 4:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [ ]:
# Function to convert celcius to fahrenheit
def convert_fahrenheit(temp):
  result = (1.8*temp)+32
  return result

## Objective
The task is to design some new features to try to improve the ability of a model to predict the total number of shared bike rentals during a specific hour of the day.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [ ]:
df.duplicated().sum()

0

**Notes:**
* No null values.
* No duplicated records.
* Need to change the datetime to it's correct data type

In [ ]:
df.drop(['casual', 'registered'], axis=1, inplace=True) # those characteristics bring no value to our objective
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,2011-01-01 0:00:00,1,0,0,1,9.84,14.395,81,0.0,16
1,2011-01-01 1:00:00,1,0,0,1,9.02,13.635,80,0.0,40
2,2011-01-01 2:00:00,1,0,0,1,9.02,13.635,80,0.0,32
3,2011-01-01 3:00:00,1,0,0,1,9.84,14.395,75,0.0,13
4,2011-01-01 4:00:00,1,0,0,1,9.84,14.395,75,0.0,1


In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [ ]:
df['datetime'].info() # the change was done correctly

<class 'pandas.core.series.Series'>
RangeIndex: 10886 entries, 0 to 10885
Series name: datetime
Non-Null Count  Dtype         
--------------  -----         
10886 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 85.2 KB


In [ ]:
df['Month'] = df['datetime'].dt.month_name()
df['Day_of_week'] = df['datetime'].dt.day_name()
df['Hour'] = df['datetime'].dt.hour

df.drop(['datetime', 'season'], axis=1, inplace=True)
df.head()

,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,Month,Day_of_week,Hour
0,0,0,1,9.84,14.395,81,0.0,16,January,Saturday,0
1,0,0,1,9.02,13.635,80,0.0,40,January,Saturday,1
2,0,0,1,9.02,13.635,80,0.0,32,January,Saturday,2
3,0,0,1,9.84,14.395,75,0.0,13,January,Saturday,3
4,0,0,1,9.84,14.395,75,0.0,1,January,Saturday,4


In [ ]:
df[['Month', 'Day_of_week', 'Hour']].info() # all three new columns need to be objects so I can ohe them 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Month        10886 non-null  object
 1   Day_of_week  10886 non-null  object
 2   Hour         10886 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 255.3+ KB


In [ ]:
df['Hour'] = df['Hour'].astype('object')
df['Hour'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 10886 entries, 0 to 10885
Series name: Hour
Non-Null Count  Dtype 
--------------  ----- 
10886 non-null  object
dtypes: object(1)
memory usage: 85.2+ KB


Las temperaturas en la columna “temp” y “atemp” son en Celsius. Utiliza .appply() para convertirlas en Fahrenheit.

In [ ]:
df['temp'] = df['temp'].apply(convert_fahrenheit)
df['atemp'] = df['atemp'].apply(convert_fahrenheit)
df.loc[:, ['temp','atemp']]

,temp,atemp
0,49.712,57.911
1,48.236,56.543
2,48.236,56.543
3,49.712,57.911
4,49.712,57.911
...,...,...
10881,60.044,67.451
10882,58.568,63.365
10883,57.092,60.638
10884,57.092,63.365


In [ ]:
df['temp_variance'] = df.apply(lambda x: x['atemp'] - x['temp'], axis=1)
df.drop('atemp', axis=1, inplace=True) # It brings no use to the objective
df.head()

,holiday,workingday,weather,temp,humidity,windspeed,count,Month,Day_of_week,Hour,temp_variance
0,0,0,1,49.712,81,0.0,16,January,Saturday,0,8.199
1,0,0,1,48.236,80,0.0,40,January,Saturday,1,8.307
2,0,0,1,48.236,80,0.0,32,January,Saturday,2,8.307
3,0,0,1,49.712,75,0.0,13,January,Saturday,3,8.199
4,0,0,1,49.712,75,0.0,1,January,Saturday,4,8.199


# Model with feature Engineering

In [126]:
y = df['count']
X = df.drop(columns='count')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   holiday        10886 non-null  int64  
 1   workingday     10886 non-null  int64  
 2   weather        10886 non-null  int64  
 3   temp           10886 non-null  float64
 4   humidity       10886 non-null  float64
 5   windspeed      10886 non-null  float64
 6   count          10886 non-null  int64  
 7   Month          10886 non-null  object 
 8   Day_of_week    10886 non-null  object 
 9   Hour           10886 non-null  object 
 10  temp_variance  10886 non-null  float64
dtypes: float64(4), int64(4), object(3)
memory usage: 935.6+ KB


In [ ]:
df['holiday'].value_counts() 

0    10575
1      311
Name: holiday, dtype: int64

In [ ]:
df['workingday'].value_counts()

1    7412
0    3474
Name: workingday, dtype: int64

In [ ]:
df['weather'].value_counts()

1    7192
2    2834
3     859
4       1
Name: weather, dtype: int64

**Notes:**
* OHE: holiday, workingday, weather, Month, Day_of_week, hour (I will count these values as categories, as they represent a yes or no, or a type of state depending on what is mentioned.)
* Scaler: temp, humidity, windspeed, temp_variance

In [127]:
df['humidity'] = df['humidity'].astype('float') # So it's easier for me to select the respective values.

In [128]:
num_selector = make_column_selector(dtype_include='float')
cat_selector = make_column_selector(dtype_include=['object', 'int'])

In [129]:
scaler = StandardScaler()
ohe = OneHotEncoder()

In [130]:
num_pipeline = make_pipeline(scaler)
num_pipeline

Pipeline(steps=[('standardscaler', StandardScaler())])

In [131]:
cat_pipeline = make_pipeline(ohe)
cat_pipeline

Pipeline(steps=[('onehotencoder', OneHotEncoder())])

In [132]:
num_tuple = (num_pipeline, num_selector)
cat_tuple = (cat_pipeline, cat_selector)

In [133]:
preprocessor = make_column_transformer(num_tuple, cat_tuple)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fb13a5738e0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('onehotencoder',
                                                  OneHotEncoder())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fb13a573070>)])

In [134]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fb13a5738e0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('onehotencoder',
                                                  OneHotEncoder())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fb13a573070>)])

In [135]:
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [136]:
X_train_processed = X_train_processed.toarray()
df_processed = pd.DataFrame(X_train_processed)
df_processed

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,1.084597,0.164369,0.025162,0.287383,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.348279,-0.510793,1.121515,0.082940,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-1.019170,-0.043373,-0.826470,0.118079,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.453467,-0.095309,-0.461834,0.044607,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.545112,1.618566,0.025162,-1.134135,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8159,-1.755488,-0.770471,-0.217928,-1.054275,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8160,-1.124358,-0.043373,0.756064,-1.293857,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8161,-1.124358,-0.718536,0.512158,-0.811499,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8162,-0.598417,-2.328540,2.704049,0.440715,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
rf = RandomForestRegressor(random_state=42, max_depth=31, n_estimators=98)
rf.fit(X_train_processed, y_train)

rf_train_preds = rf.predict(X_train_processed)
rf_test_preds = rf.predict(X_test_processed)

print('R2 train score of:', r2_score(y_train, rf_train_preds)*100)
print('R2 test score of:', r2_score(y_test, rf_test_preds)*100)
print('\n')
print('The MAE of train is: ', mean_absolute_error(y_train, rf_train_preds))
print('The MAE of test is: ', mean_absolute_error(y_test, rf_test_preds))
print('\n')
print('The RMSE of train is: ', np.sqrt(mean_squared_error(y_train, rf_train_preds)))
print('The RMSE of test is: ', np.sqrt(mean_squared_error(y_test, rf_test_preds)))

R2 train score of: 97.5069134064867
R2 test score of: 84.29624507823794


The MAE of train is:  19.022303861497406
The MAE of test is:  48.40403437270257


The RMSE of train is:  28.599870436700733
The RMSE of test is:  71.78465321047658


In [ ]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [ ]:
param_grid = [
  {'n_estimators': list(range(90,100, 2)), 
   'max_depth': list(range(25, 35, 3))}
]

rndf_reg = RandomForestRegressor()

grid_search = GridSearchCV(rndf_reg, param_grid, cv=5)

grid_search.fit(X_train_processed, y_train)

grid_search.best_params_

{'max_depth': 31, 'n_estimators': 98}

In [140]:
bagreg = BaggingRegressor(random_state=42, n_estimators=9)
bagreg.fit(X_train_processed, y_train)

bagreg_train_pred = bagreg.predict(X_train_processed)
bagreg_test_pred = bagreg.predict(X_test_processed)

print('R2 train score of:', r2_score(y_train, bagreg_train_pred)*100)
print('R2 test score of:', r2_score(y_test, bagreg_test_pred)*100)
print('\n')
print('The MAE of train is: ', mean_absolute_error(y_train, bagreg_train_pred))
print('The MAE of test is: ', mean_absolute_error(y_test, bagreg_test_pred))
print('\n')
print('The RMSE of train is: ', np.sqrt(mean_squared_error(y_train, bagreg_train_pred)))
print('The RMSE of test is: ', np.sqrt(mean_squared_error(y_test, bagreg_test_pred)))

R2 train score of: 96.39691952983878
R2 test score of: 81.85876671735379


The MAE of train is:  20.712705509281943
The MAE of test is:  51.43315780880072


The RMSE of train is:  34.3820972865616
The RMSE of test is:  77.1548544371092


In [138]:
bagreg.get_params()

{'base_estimator': 'deprecated',
 'bootstrap': True,
 'bootstrap_features': False,
 'estimator': None,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [139]:
param_grid = [
  {'n_estimators': list(range(1, 10))}
]

bag_reg = BaggingRegressor(random_state=42)

grid_search = GridSearchCV(bag_reg, param_grid, cv=5)

grid_search.fit(X_train_processed, y_train)

grid_search.best_params_

{'n_estimators': 9}